In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sortedcontainers import SortedList
from IPython.display import clear_output

In [2]:
# Load data
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv').drop('timestamp', axis=1)

In [3]:
# Create a mapper from movieId to movie title
movie2title = dict(zip(movies['movieId'], movies['title']))

# Create a mapper from index to id
user2idx = dict(zip(ratings['userId'].unique(), range(ratings['userId'].nunique())))
movie2idx = dict(zip(ratings['movieId'].unique(), range(ratings['movieId'].nunique())))

# Create a mapper from id to idex
idx2user = dict(zip(range(ratings['userId'].nunique()), ratings['userId'].unique()))
idx2movie = dict(zip(range(ratings['movieId'].nunique()), ratings['movieId'].unique()))

# Convert id to index in dataframe
ratings['userId'] = ratings['userId'].apply(lambda x: user2idx[x])
ratings['movieId'] = ratings['movieId'].apply(lambda x: movie2idx[x])

# Shuffle the ratings table
data = shuffle(ratings)

# Init ratio to split the data: 80%, 20%
ratio = int(0.8*len(data))

# Split into train_set and test_set
train_set = data[:ratio].reset_index().drop('index', axis=1)
test_set = data[ratio:].reset_index().drop('index', axis=1)

In [4]:
print("Training set size:", train_set.shape[0])
print("Test set size:", test_set.shape[0])

Training set size: 80668
Test set size: 20168


In [5]:
# a dictionary tell us which users have rated which movies
user2movie = {}

# a dictionary tell us which movies have been rated by which users
movie2user = {}

# a dictionary to look up ratings based on userId and movieId
usermovie2rating = {}

for i in range(len(train_set)):
    # user2movie
    if train_set['userId'][i] not in user2movie:
        user2movie[train_set['userId'][i]] = [train_set['movieId'][i]]
    else:
        user2movie[train_set['userId'][i]].append(train_set['movieId'][i])
    
    # movie2user
    if train_set['movieId'][i] not in movie2user:
        movie2user[train_set['movieId'][i]] = [train_set['userId'][i]]
    else:
        movie2user[train_set['movieId'][i]].append(train_set['userId'][i])
        
    # usermovie2rating
    usermovie2rating[(train_set['userId'][i], train_set['movieId'][i])] = train_set['rating'][i]
    
# a dictionary to look up ratings based on userId and movieId (test set)
usermovie2rating_test = {}

for i in range(len(test_set)):
    usermovie2rating_test[(test_set['userId'][i], test_set['movieId'][i])] = test_set['rating'][i]
    
# utility matrix
U = train_set.pivot_table(values='rating', index='userId', columns='movieId') 

# to handle if training set doesn't have userId or movieId that test set has
for userId in (set(test_set['userId']) - set(train_set['userId'])):
    U.append(pd.Series(name=userId), ignore_index=False)
for movieId in (set(test_set['movieId']) - set(train_set['movieId'])):
    U[movieId] = np.nan

# raw average rating (mean of all the ratings)
r = train_set['rating'].mean()
# movie bias
b_i = U.mean() - r
# user bias
b_u = U.mean(axis=1) - r

In [6]:
def cosine_similarity(r1, r2):
    """Return the cosine similarity between two rating arrays"""
    return (r1.dot(r2))/(np.linalg.norm(r1) * np.linalg.norm(r2)) 

k = 20 # number of nearest neighbors
limit = 5 # number of movies users must have in common in order to consider
n_users = ratings['userId'].nunique()
neighbors = [] # to store neighbors

for u in range(n_users):
    # list of movies that user u has rated
    movie_u = user2movie[u] 
    
    # to store similarity between user u and other users
    sim_list = SortedList()
    
    for v in range(n_users):
        if u != v:
            # list of movies that user u has rated
            movie_v = user2movie[v]
            
            # list of common movies between user u and v
            common = set(movie_u) & set(movie_v)
            
            if len(common) > limit:
                # users' rating vector 
                r_u = np.array([usermovie2rating[(u, i)] for i in common])
                r_v = np.array([usermovie2rating[(v, i)] for i in common])
                
                
                # similarity between user u and v
                sim_uv = cosine_similarity(r_u, r_v)
                
                # only add positive similarity
                if sim_uv > 0:
                    sim_list.add((-sim_uv, v))
                    
    neighbors.append(sim_list)
    
    clear_output(wait=True)
    print('Calculate k nearest neighbors for each user: ' + str(round(((u+1)/n_users)*100, 2)) + '%')

Calculate k nearest neighbors for each user: 100.0%


In [7]:
def predict(u, i):
    # base line prediction
    b_ui = r + b_i[i] + b_u[u]
    
    # to calculate number of neighbors 
    count_k = 0
    
    # innit numerator and denominator of the bias
    numerator = 0
    denominator = 0
    for neg_sim_uv, v in neighbors[u]:
        if count_k < k:
            try:
                b_vi = r + b_i[i] + b_u[v]
                numerator += (-neg_sim_uv * (usermovie2rating[(v, i)] - b_vi))
                denominator += (-neg_sim_uv)
                count_k += 1
            except:
                pass
    
    # in case there is no neighbor for user u, just use the baseline prediction
    if denominator == 0:
        prediction = r + b_i[i] + b_u[u]
    else:
        prediction = b_ui + numerator / denominator
    
    # incase the prediction is out of scale
    prediction = min(5, prediction)
    prediction = max(0.5, prediction)    
    
    return prediction    

In [8]:
# use to evaluate performance on training set
train_prediction = []
train_actual_rating = []

# use to display status bar
c = 0
C = len(usermovie2rating)

for (u, i), rating in usermovie2rating.items():
    c += 1
    # add actual rating to list
    train_actual_rating.append(rating)
    
    # add predicted rating to list
    train_prediction.append(predict(u, i))
    
    clear_output(wait=True)
    print('Calculate prediction for training set: ' + str(round(((c+1)/C)*100, 2)) + '%')

Calculate prediction for training set: 100.0%


In [9]:
# use to evaluate performance on test set
test_prediction = []
test_actual_rating = []

# use to display status bar
c = 0
C = len(usermovie2rating_test)

for (u, i), rating in usermovie2rating_test.items():
    c += 1
    # add actual rating to list
    test_actual_rating.append(rating)
    
    # add predicted rating to list
    test_prediction.append(predict(u, i))
    
    clear_output(wait=True)
    print('Calculate prediction for test set: ' + str(round(((c+1)/C)*100, 2)) + '%')

Calculate prediction for test set: 100.0%


In [10]:
def rmse(x1, x2):
    x1 = np.array(x1)
    x2 = np.array(x2)
    return np.sqrt(np.mean((x1 - x2) ** 2))

print("Training rmse:", rmse(train_actual_rating, train_prediction))
print("Test rmse:", rmse(test_actual_rating, test_prediction))

Training rmse: 0.8359563697536861
Test rmse: 0.9688988492394248


In [11]:
def get_recommendation(u):
    # list of movieId that user u hasn't watched yet
    movie_u = set(ratings['movieId']) - set(train_set[train_set['userId'] == u]['movieId'].unique())
    
    # list of prediction
    candidate = SortedList()
    
    # use to display status bar
    c = 0
    C = len(movie_u)
    
    for i in movie_u:
        c += 1
        prediction = predict(u, i)
        candidate.add((-prediction, i))
        
        clear_output(wait=True)
        print(str(round(((c+1)/C)*100, 2)) + '%')
    
    clear_output(wait=True)
    # print top 10 recommendation for user u
    print("Recommendation for user " + str(u) + ":")
    print("____________________________________________________________")
    for (p, i) in candidate[:10]:
        print(movie2title[idx2movie[i]])

In [12]:
get_recommendation(2)

Recommendation for user 2:
____________________________________________________________
Galaxy of Terror (Quest) (1981)
Master of the Flying Guillotine (Du bi quan wang da po xue di zi) (1975)
Nobody Loves Me (Keiner liebt mich) (1994)
Cure, The (1995)
Gordy (1995)
Rent-a-Kid (1995)
The Hundred-Foot Journey (2014)
Green Butchers, The (Grønne slagtere, De) (2003)
Mesrine: Killer Instinct (L'instinct de mort) (2008)
White Ribbon, The (Das weiße Band) (2009)
